<h1>Preprocessing</h1>

In [21]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [22]:
books = pd.read_csv('data/books.csv', on_bad_lines = 'skip')

In [23]:
books.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


In [24]:
books.shape

(11123, 12)

In [25]:
books.isnull().any()

bookID                False
title                 False
authors               False
average_rating        False
isbn                  False
isbn13                False
language_code         False
  num_pages           False
ratings_count         False
text_reviews_count    False
publication_date      False
publisher             False
dtype: bool

In [26]:
books.dtypes

bookID                  int64
title                  object
authors                object
average_rating        float64
isbn                   object
isbn13                  int64
language_code          object
  num_pages             int64
ratings_count           int64
text_reviews_count      int64
publication_date       object
publisher              object
dtype: object

In [27]:
le = LabelEncoder()
for col in books.columns:
    if books[col].dtype == 'object':
        books[col] = le.fit_transform(books[col])

books.dtypes

bookID                  int64
title                   int64
authors                 int64
average_rating        float64
isbn                    int64
isbn13                  int64
language_code           int64
  num_pages             int64
ratings_count           int64
text_reviews_count      int64
publication_date        int64
publisher               int64
dtype: object

In [28]:
books.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,3070,2626,4.57,4113,9780439785969,5,652,2095690,27591,3440,1778
1,2,3071,2626,4.49,4051,9780439358071,5,870,2153167,29221,3342,1778
2,4,3068,2622,4.42,4079,9780439554893,5,352,6333,244,719,1773
3,5,3073,2626,4.56,4095,9780439655484,5,435,2339585,36325,2186,1778
4,8,3063,2626,4.78,4100,9780439682589,5,2690,41428,164,3397,1773


<h1>Visualizations</h1>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sb

<h1>Modeling</h1>

In [29]:
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA

